In [ ]:
### 声发射率
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import csv
import torch 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 

# file_name=os.listdir("result2")
strs="144051"
strss="144051"
file = pd.read_csv(strss+"_filename.csv",header=None, encoding="utf-8").values
chss=['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8','ch9','ch10','ch11','ch12'] 

files=[]
num=[]
for inxs in range(file.shape[0]):  ##file.shape[0]
    name = str(file[inxs][0])
    folders = os.path.join("result_fast_fig", name)
    if not  os.path.exists(folders):
        os.makedirs(folders)
    pt=[]
    for i in range(12):
        file_name = chss[i]+"_"+name+".mat"  #12个文件名字
        # print(file_name)
        path = os.path.join("result_pick",strss, file_name).replace("\\","/")    #12个通道的路径
        data_=sio.loadmat(path)["data"]                  #12个通道的数据   1，点数，信噪比， 概率
        data = np.sort(sio.loadmat(path)["data"][:,1])   #按从小到大的顺序进行排列
        pt.append(data.astype(int))                      #12个通道的初至点
        narry = np.zeros([len(pt),len(max(pt,key = lambda x: len(x)))])   #
        for i, j in enumerate(pt):
            narry[i][0:len(j)] = j   #(12, n)
        point=narry.reshape(-1)
        pts=np.sort(point[point!=0])
        ptime = pts
        
    event=[]  #单个通道含有的事件
    while True:
        if pts.shape[0] < 1:
            break
        ref = pts[0]
        event.append(pts[np.abs(ref-pts)<=750])
        pts=pts[np.abs(ref-pts)>750]
    
    events=[]  #存在两个通道含有 
    count=0
    for idx in range(len(event)):
        if len(event[idx])>= 3:
            count += 1
            events.append(event[idx])
    # num =0+count  
    chs=[]   #找出对应初至点的通道
    for ind in range(len(events)):
        et = events[ind]
        ch = np.zeros(len(et))
        for inx in range(len(et)):
            ch[inx]=np.argwhere(narry==et[inx])[0,0]  
        chs.append(ch)
        
    files.append(name)
    num.append(len(events))
    filess=np.array(files)
    nums = np.array(num)
    datas = np.vstack((filess,nums))
    print(len(chs),len(events),len(nums), len(filess))
    
    for i in range(len(events)): #所有的事件len(events)
        ets = events[i]   #单个事件
        cns = chs[i]      #单个事件的不同通道
        toto =[]
        for j in range(len(cns)): #len(cns)
            pn = int(ets[j])
            cn = int(cns[j]) 
            filename_ = chss[cn]+"_"+name+".mat"
            # print(filename_)
            pth = os.path.join("G:\labeq_20220315", chss[cn], filename_)
            data = sio.loadmat(pth)["data"][:,0]
            if pn < 2000:
                rdata = data[pn-pn:pn+8000-pn]
            elif pn > 2994000:
                rdata = data[pn-(8000-3000000+pn):3000000]
            else:
                rdata = data[pn-2000:pn+6000]
            rdata = rdata-np.mean(rdata)
            rdata = rdata/(np.max(np.abs(rdata))+1e-6)
            toto.append(rdata)
            # print(pth)
        names = name + "_"+ str(i)
        # fig_path=os.path.join(folders, names).replace("\\","/") 
        # gs = gridspec.GridSpec(len(cns), 1)
        # fig = plt.figure(1, figsize=(16, 16), dpi=100) 
        # for k in range(len(cns)):
        #     ax = fig.add_subplot(gs[k, 0])
        #     ax.plot(toto[k], alpha=0.5, c="b")
        #     ax.vlines(2000, -0.8, 0.8, "r")
        # plt.savefig(fig_path)
        # plt.cla() 
        # plt.clf()
    with open('ckpt/'+strss+'/events.csv','a', newline='') as f:
        writer=csv.writer(f)
        writer.writerow("E"+str(inxs)) #+str(inxs)
        for l2 in events:
            writer.writerow(l2)
    with open('ckpt/'+strss+'/chs.csv','a', newline='') as f:
        writer=csv.writer(f)
        writer.writerow("E"+str(inxs)) #+str(inxs)
        for l2 in chs:
            writer.writerow(l2)
    
sio.savemat("odata/"+strss+"/AE_rate.mat", {"data": datas})

In [2]:
### 路径、初至点、时间
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import csv
import torch 
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator 

strs="144051"
path_ = "G:\labeq_20220315"
chs=['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8','ch9','ch10','ch11','ch12'] 

event_ = open("ckpt/"+strs+"/events.csv","r", encoding="utf-8")
chn_ = open("ckpt/"+strs+"/chs.csv","r", encoding="utf-8")
file_name = np.array(pd.read_csv(strs+"_filename.csv", header=None, encoding="utf-8"))[:,0]

## 初至点的时间 0~140s
info = {}
point = {}
for line in event_.readlines():
    sline = [i.replace(" ", "") for i in line.strip().split(",")]  #每行的数据
    if sline[0] == "E":
        time = []
        pt = []
        toto="0"
        for i in range(len(sline)):
            toto = toto+sline[i]
        to = int(float(toto[2:]))
    elif sline[0] !="E":
        for inx in sline:
            t = to + int(float(inx))/3000000
            time.append(t)
            pt.append(int(float(inx)))
    info[to] = time
    point[to] = pt

times=[]
for idx in range(len(info)):
    times.extend(info[idx])
    
points=[]
for idx in range(len(point)):
    points.extend(point[idx])

##拾取事件的路径
infos={}
for line in chn_.readlines():
    slines = [i.replace(" ", "") for i in line.strip().split(",")]
    if slines[0] == "E":
        paths = []
        toto = "0"
        for i in range(len(slines)):
            toto = toto+slines[i]
        to = int(float(toto[2:]))
    elif slines[0] !="E":
        for inx in slines:
            chn = chs[int(float(inx))]
            file_ = file_name[to]
            filename = chn+"_"+str(file_)+".mat"
            path = os.path.join(path_, chn, filename).replace("\\","/")
            paths.append(path)
    infos[to]=paths
            
phs=[]
for idx in range(len(infos)):
    phs.extend(infos[idx])
print("time:",len(times), "point:", len(points), "path:", len(phs))

times = np.array(times)
points = np.array(points)
phs = np.array(phs)
data = np.vstack((phs, points, times))

np.savetxt("ckpt/"+strs+"/pick_result.csv", data, delimiter=",", fmt="%s" )

time: 15910 point: 15910 path: 15910


In [3]:
### 时间，震级
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import csv
import torch 
import matplotlib.pyplot as plt

strs="144051"
event_ = open("ckpt/"+strs+"/events.csv","r", encoding="utf-8")
data=np.array(pd.read_csv("ckpt/"+strs+"/pick_result.csv", header=None, encoding="utf-8"))  #(3*n)

nx=0
ts =[]
m = []
dB = []
for line in event_.readlines():
    sline = [i.replace(" ", "") for i in line.strip().split(",")]  #每行的数据
    if sline[0] == "E":
        toto = "0"
        for i in range(len(sline)):
            toto = toto+sline[i]
        head = int(float(toto[2:]))  #开头
    elif sline[0] != "E":
        num = len(sline)    #每行的数量
        path = data[0][nx:nx+num]  #每行的路径
        pt = data[1][nx:nx+num]    #每行的初至点
        time = data[2][nx:nx+num]  #每行的时间
        nx = nx+num
        
        #每个事件的时间  取平均
        times=[]
        for j in range(num):    #每行的时间点 取平均  1510个事件点
            times.append(float(time[j]))
        t = np.mean(times)
        ts.append(t)
        
        #每个事件的震级 取平均
        amplitude = []
        for j in range(num):    #每行的数据点
            pts = int(pt[j])
            phs = path[j]
            toto = sio.loadmat(phs)["data"][:,0]
            toto = toto - np.mean(toto)
            if pts < 100 :
                rdata = toto[pts-pts:pts+6000-pts]
            elif pts > 2994000:
                rdata = toto[pts-(4000-3000000+pts):3000000]
            else:
                rdata = toto[pts-100:pts+6000]
            # rdata = rdata - np.mean(rdata)
            # signal = rdata/ np.max(np.abs(rdata)+1e-6)
            amplitude.append(np.max(rdata))
        amp_ave= np.mean(amplitude)
        mag=np.log10(amp_ave) 
        AdB=20*np.log10(amp_ave) 
        
        m.append(mag)
        dB.append(AdB)  

ts = np.array(ts)
m = np.array(m)
dB = np.array(dB)
data = np.vstack((ts, m, dB))
np.savetxt("ckpt/"+strs+"/mag_result.csv", data, delimiter=",", fmt="%s" )

In [4]:
##计算模型的总体时间 CPU
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import csv
import torch 
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 12
plt.switch_backend("agg") 
plt.rcParams['font.sans-serif'] = 'times new roman'
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.major.size']  = 8
plt.rcParams['xtick.minor.size']  = 4
plt.rcParams['ytick.major.size']  = 8
plt.rcParams['ytick.minor.size']  = 4

time = pd.read_csv("time_144836_1.csv", header=None, encoding="utf-8").values

ave_time = np.mean(time)
sum_time = np.sum(time)
print(ave_time,sum_time)

##不同时间的增加数量
data =  sio.loadmat("odata/144836/AE_rate.mat", )["data"]
nums=[]
sums=[]
for i in data[1]:
    nums.append(int(i.strip()))
for j in range(0,7):
    toto = np.sum(nums[0+20*j:20+20*j])
    sums.append(toto)
print(sums)
    
colors = ["#ff0000", "#0000ff"]

fig, ax = plt.subplots(figsize=(9,6), dpi=100)
ax.bar(range(len(sums)), sums, color=colors[0], alpha=0.25,edgecolor=colors[0])
ax.tick_params(axis='both', labelsize=20)
# ax.set_xlabel('Time Residuals (ms)', fontsize=26)
# ax.set_ylabel('Number of Picks', fontsize=26)

18.099190250999975 21719.02830119997
[34, 31, 22, 1016, 884, 292, 249]
